## Task  1: Loading Transactions from a CSV File

In [674]:
import csv
from datetime import datetime
import random

In [675]:
def load_transactions(filename='financial_transactions.csv'):
    """Load transactions from a CSV file into a list of dictionaries."""
    transactions = []
    try:
        # Open file with 'with' statement
        with open((filename), 'r') as file:
        # Use csv.DictReader
            reader = csv.DictReader(file)
        # For each row:
            for row in reader:
        #   Parse date with datetime.strptime
                row['date'] = datetime.strptime(row['date'], '%Y-%m-%d').date()
        #   Make amount negative for 'debit'
                if row['type'] == 'debit':
                    row['amount'] = -float(row['amount'])
                else:
                    row['amount'] = float(row['amount'])
        #   Create dictionary with all fields
                transaction = {
                    'transaction_id': row['transaction_id'],
                    'date': row['date'],
                    'customer_id': row['customer_id'],
                    'amount': row['amount'],
                    'type': row['type'],
                    'description': row['description'],
                }
        #   Add to transactions
                transactions.append(transaction)
    # Catch FileNotFoundError, ValueError
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
    except ValueError as e:
        print(f"Error: {e}. Please check the date format in the file.")
    
    return transactions


    

### Additional tasks to be completed
1. Print the number of loaded transactions.
2. Skip rows with invalid amounts and print a warning.
3. Test with a CSV containing one bad date and verify it’s skipped.
4. Log errors to `errors.txt`.

In [676]:
# Print the number of loaded transactions
print (len(load_transactions()))



100000


In [677]:
# Skip rows with invalid amounts and print a warning.
def load_transactions(filename='financial_transactions.csv'):
    """Load transactions from a CSV file into a list of dictionaries."""
    transactions = []
    try:
        # Open file with 'with' statement
        with open((filename), 'r') as file:
        # Use csv.DictReader
            reader = csv.DictReader(file)
        # For each row:
            for row in reader:
        #   Parse date with datetime.strptime
                # Skip rows with invalid dates
                try:
                    row['date'] = datetime.strptime(row['date'], '%Y-%m-%d').date()
                except ValueError:
                    print("Warning, invalid date detected, skipping row.")
                    # Log errors to errors.txt
                    with open("errors.txt", "w") as file:
                        file.write(f"Warning, invalid date detected in row: {row}\n")
                    continue
        #   Make amount negative for 'debit'
                try:
                    if row['type'] == 'debit':
                        row['amount'] = -float(row['amount'])
                    else:
                        row['amount'] = float(row['amount'])
                except ValueError:
                    print("Warning, invalid amount detected, skipping row.")
                    # Log errors to errors.txt
                    with open("errors.txt", "a") as file:
                        file.write(f"Warning, invalid amount detected in row: {row}\n")
                    continue
        #   Create dictionary with all fields
                transaction = {
                    'transaction_id': row['transaction_id'],
                    'date': row['date'],
                    'customer_id': row['customer_id'],
                    'amount': row['amount'],
                    'type': row['type'],
                    'description': row['description'],
                }
        #   Add to transactions
                transactions.append(transaction)
    # Catch FileNotFoundError, ValueError
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
        # Log errors to errors.txt
        with open("errors.txt", "a") as file:
            file.write(f"Error: The file {filename} was not found.")
    except ValueError as e:
        print(f"Error: {e}. Please check the date format in the file.")
        # Log errors to errors.txt
        with open("errors.txt", "a") as file:
            file.write(f"Error: {e}. Please check the date format in the file.")
    # Print the number of loaded transactions
    print(f"Number of loaded transactions: {len(transactions)}")
    return transactions




## Task 2: Adding and Viewing Transactions

### Additional tasks to be completed
1. Reject invalid transaction types in `add_transaction`.
2. Add an option to `view_transactions` to filter by type.
3. Suggest customer IDs from existing transactions.
4. Format dates as “Oct 26, 2020” in `view_transactions`.


### For adding:

In [678]:
def add_transaction(transactions):
    """Add a new transaction from user input."""
    # Prompt for date, customer_id, amount, type, description
    
    # Validate date
    while True:
        try:
            date = datetime.strptime(input("Enter date (YYYY-MM-DD): "), '%Y-%m-%d').date()
            break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")
            continue
    # Suggest customer_id from random number not in transactions
    # Get existing customer IDs
    existing_ids = {t['customer_id'] for t in transactions}
    
    # Generate potential new ID up to max + 1
    max_id = len(transactions) + 1
    while True:
        suggested_id = random.randint(1, max_id)
        if suggested_id not in existing_ids:
            break
    
    # Suggest and get customer_id
    print(f"Suggested customer ID: {suggested_id}")
    customer_id = input(f"Enter customer ID (press Enter to use suggestion): ").strip()
    if not customer_id:
        customer_id = suggested_id
    # customer_id = input("Enter customer ID: ")
    
    # Validate amount
    while True:
        try:
            amount = float(input("Enter amount: "))
            break
        except ValueError:
            print("Invalid amount. Please enter a number.")
            continue
    # Validate type
    while True:
        type = input("Enter type (credit/debit/transfer): ").lower()
        if type == 'debit':
            amount = -abs(amount)  # Make amount negative for debit
            break
        elif type == 'credit':
            amount = abs(amount)
            break
        elif type == 'transfer':
            # For transfer, we can keep the amount positive
            amount = abs(amount)
            break
        # reject invalid transaction types
        else:
        # elif type not in ['credit', 'transfer']:
            print("Invalid type. Please enter 'credit', 'debit', or 'transfer'.")
            continue
    description = input("Enter description: ")
  
    # Generate new transaction_id
    transaction_id = len(transactions) + 1  # ID generation based on current length
    # Create dictionary and append
    transaction = {
        'transaction_id': transaction_id,
        'date': date,
        'customer_id': customer_id,
        'amount': amount,
        'type': type,
        'description': description
    }
    transactions.append(transaction)
    print(f"Transaction {transaction_id} added successfully.")
    return transactions



### For viewing:

In [679]:
def view_transactions(transactions, transaction_type=None):
    """Display transactions in a table."""
    # Print header
    header = f"{'ID':^5} {'Date':^12} {'Customer':^12} {'Amount':^10} {'Type':^10} {'Description':^30}"
    print('-' * 83)  # Print a line of dashes
    print(header)
    print('-' * 83)  # Print a line of dashes
    print()
    # Filter by type if specified
    displayed_transactions = transactions
    if transaction_type:
        displayed_transactions = [t for t in transactions if t['type'] == transaction_type]
    # Loop through transactions
    for t in transactions:
        # Format amount with 2 decimal places and comma
        amount_str = f"{t['amount']:,.2f}"
        # Format each row
        row = (f"{str(t['transaction_id']):^5} "    # Center ID
                # Center date and display in 'MMM. DD, YYYY' format
               f"{t['date'].strftime('%b. %d, %Y'):^15} "
               f"{t['customer_id']:^12} "           # Center customer ID
               f"{amount_str:<10} "                 # Left justify amount
               f"{t['type']:^10} "                  # Center type
               f"{t['description']:<30}")           # Left justify description
        # Print each row
        print(row)
    # Print footer
    print('-' * 83)  # Print a line of dashes
    return transactions



## Task 3: Updating and Deleting Transactions

### For updating:

In [680]:
def update_transaction(transactions):
    """Update a transaction’s details."""
    # Show transactions with numbers
    view_transactions(transactions)
    # Ask user to pick a number
    while True:
        try:
            transaction_id = int(input("Enter the transaction ID to update: "))
            # Check if ID exists
            if any(int(t['transaction_id']) == transaction_id for t in transactions):
                break
            else:
                print("Transaction ID not found. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a valid transaction ID.")
    # Ask which field to change
    while True:
        field = input("Which field would you like to update? (date, customer_id, amount, type, description): ").strip().lower()
        if field in ['date', 'customer_id', 'amount', 'type', 'description']:
            break
        else:
            print("Invalid field. Please enter one of: date, customer_id, amount, type, description.")
    # Update field
    for t in transactions:
        if int(t['transaction_id']) == transaction_id:
            if field == 'date':
                while True:
                    try:
                        new_date = datetime.strptime(input("Enter new date (YYYY-MM-DD): "), '%Y-%m-%d').date()
                        t['date'] = new_date
                        break
                    except ValueError:
                        print("Invalid date format. Please use YYYY-MM-DD.")
            elif field == 'customer_id':
                new_customer_id = input("Enter new customer ID: ").strip()
                t['customer_id'] = new_customer_id
            elif field == 'amount':
                while True:
                    try:
                        new_amount = float(input("Enter new amount: "))
                        t['amount'] = new_amount
                        break
                    except ValueError:
                        print("Invalid amount. Please enter a number.")
            elif field == 'type':
                while True:
                    new_type = input("Enter new type (credit/debit/transfer): ").lower()
                    if new_type in ['credit', 'debit', 'transfer']:
                        t['type'] = new_type
                        if new_type == 'debit':
                            t['amount'] = -abs(t['amount'])  # Make amount negative for debit
                        else:
                            t['amount'] = abs(t['amount'])  # Keep positive for credit/transfer
                        break
                    # Prevent invalid type updates
                    else:
                        print("Invalid type. Please enter 'credit', 'debit', or 'transfer'.")
            elif field == 'description':
                new_description = input("Enter new description: ")
                t['description'] = new_description
            print(f"Transaction {transaction_id} updated successfully.")
            break
    # Show updated transactions
    view_transactions(transactions)    

    return transactions

### Additional tasks to be completed
1. Prevent invalid type updates in `update_transaction`.
2. Allow updating multiple fields at once.
3. Add a cancel option for updates/deletions.

#### Update multiple fields and cancel version

In [681]:
def update_transaction(transactions):
    """Update a transaction's details."""
    # Show transactions with numbers
    view_transactions(transactions)
    # Ask user to pick a number or cancel
    while True:
        choice = input("Enter the transaction ID to update (or 'cancel' to abort): ").strip().lower()
        if choice == 'cancel':
            print("Operation cancelled.")
            return transactions
        try:
            transaction_id = int(choice)
            if any(int(t['transaction_id']) == transaction_id for t in transactions):
                break
            else:
                print("Transaction ID not found. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a valid transaction ID or 'cancel'.")
    
    # Show fields available for update
    fields = ['date', 'customer_id', 'amount', 'type', 'description']
    print("\nAvailable fields for update:")
    print(", ".join(fields))
    fields_to_update = input("Enter fields to update (separate by comma), 'all', or 'cancel': ").strip().lower()
    
    # Handle cancel option
    if fields_to_update == 'cancel':
        print("Operation cancelled.")
        return transactions
    
    # Handle 'all' option or split input into list
    if fields_to_update == 'all':
        fields_to_update = fields
    else:
        fields_to_update = [f.strip() for f in fields_to_update.split(',')]
        # Validate fields
        invalid_fields = [f for f in fields_to_update if f not in fields]
        if invalid_fields:
            print(f"Invalid fields: {', '.join(invalid_fields)}")
            return transactions

    # Update fields
    for t in transactions:
        if int(t['transaction_id']) == transaction_id:
            for field in fields_to_update:
                # Add cancel option for each field update
                print(f"\nUpdating {field} (type 'cancel' to skip):")
                if field == 'date':
                    while True:
                        value = input("Enter new date (YYYY-MM-DD): ").strip()
                        if value == 'cancel':
                            print(f"Skipping {field} update.")
                            break
                        try:
                            new_date = datetime.strptime(value, '%Y-%m-%d').date()
                            t['date'] = new_date
                            break
                        except ValueError:
                            print("Invalid date format. Please use YYYY-MM-DD.")
                elif field == 'customer_id':
                    value = input("Enter new customer ID: ").strip()
                    if value != 'cancel':
                        t['customer_id'] = value
                    else:
                        print(f"Skipping {field} update.")
                elif field == 'amount':
                    while True:
                        value = input("Enter new amount: ").strip()
                        if value == 'cancel':
                            print(f"Skipping {field} update.")
                            break
                        try:
                            new_amount = float(value)
                            t['amount'] = new_amount if t['type'] != 'debit' else -abs(new_amount)
                            break
                        except ValueError:
                            print("Invalid amount. Please enter a number.")
                elif field == 'type':
                    while True:
                        value = input("Enter new type (credit/debit/transfer): ").strip().lower()
                        if value == 'cancel':
                            print(f"Skipping {field} update.")
                            break
                        if value in ['credit', 'debit', 'transfer']:
                            t['type'] = value
                            if value == 'debit':
                                t['amount'] = -abs(t['amount'])
                            else:
                                t['amount'] = abs(t['amount'])
                            break
                        else:
                            print("Invalid type. Please enter 'credit', 'debit', or 'transfer'.")
                elif field == 'description':
                    value = input("Enter new description: ").strip()
                    if value != 'cancel':
                        t['description'] = value
                    else:
                        print(f"Skipping {field} update.")
            
            print(f"\nTransaction {transaction_id} updated successfully.")
            print("\nUpdated transaction details:")
            view_transactions([t])
            break

    return transactions

### For deleting

In [682]:
def delete_transaction(transactions):
    """Delete a transaction."""
    # Show transactions with numbers
    view_transactions(transactions)
    # Ask user to pick a number
    while True:
        try:
            transaction_id = int(input("Enter the transaction ID to delete: "))
            # Find index of transaction with matching ID
            index = next(i for i, t in enumerate(transactions) if int(t['transaction_id']) == transaction_id)
            print("\nTransaction to be deleted:")
            view_transactions([transactions[index]])  # Show single transaction using array index
            break
        except ValueError:
            print("Invalid input. Please enter a valid transaction ID.")
    # Confirm and remove
    conf = input(f"Are you sure you want to delete transaction {transaction_id}? (yes/no): ").strip().lower()
    if conf == 'yes':
        transactions = [t for t in transactions if int(t['transaction_id']) != transaction_id]
        print(f"Transaction {transaction_id} deleted successfully.")
    else:
        print("Deletion cancelled.")
    # Show updated transactions
    view_transactions(transactions)
    return transactions



#### Additional tasks to be completed
1. Show transaction details before deletion.
2. Add a cancel option for updates/deletions

##### Cancelable delete

In [683]:
def delete_transaction(transactions):
    """Delete a transaction."""
    # Show transactions with numbers
    view_transactions(transactions)
    # Ask user to pick a number or cancel
    while True:
        choice = input("Enter the transaction ID to delete (or 'cancel' to abort): ").strip().lower()
        if choice == 'cancel':
            print("Operation cancelled.")
            return transactions
        try:
            transaction_id = int(choice)
            # Find index of transaction with matching ID
            index = next(i for i, t in enumerate(transactions) if int(t['transaction_id']) == transaction_id)
            print("\nTransaction to be deleted:")
            view_transactions([transactions[index]])  # Show single transaction using array index
            break
        except (ValueError, StopIteration):
            print("Invalid input. Please enter a valid transaction ID or 'cancel'.")
    
    # Confirm and remove
    conf = input("Are you sure you want to delete this transaction? (yes/no): ").strip().lower()
    if conf == 'yes':
        transactions = [t for t in transactions if int(t['transaction_id']) != transaction_id]
        print(f"Transaction {transaction_id} deleted successfully.")
    else:
        print("Deletion cancelled.")
    
    # Show updated transactions
    view_transactions(transactions)
    return transactions

## Task 4: Analyzing Financial Data

In [684]:
def analyze_finances(transactions, year=2022):
    """Calculate and display financial summaries."""
    # Filter transactions by year if specified
    if year:
        transactions = [t for t in transactions if t['date'].year == year]
        if not transactions:
            print(f"No transactions found for year {year}")
            return
        print(f"\nFinancial Analysis for {year}")
    # Sum credits, debits, transfers
    total_credit = sum(t['amount'] for t in transactions if t['type'] == 'credit')
    total_debit = sum(t['amount'] for t in transactions if t['type'] == 'debit')
    total_transfer = sum(t['amount'] for t in transactions if t['type'] == 'transfer')
    net_balance = total_credit + total_debit + total_transfer
    # Calculate total volume (absolute sum)
    total_volume = abs(total_credit) + abs(total_debit) + abs(total_transfer)
    # Print results with percentages
    print("Financial Summary:")
    if total_volume > 0:  # Avoid division by zero
        print(f"Total Credits: ${total_credit:,.2f} ({abs(total_credit/total_volume*100):.1f}%)")
        print(f"Total Debits: ${total_debit:,.2f} ({abs(total_debit/total_volume*100):.1f}%)")
        print(f"Total Transfers: ${total_transfer:,.2f} ({abs(total_transfer/total_volume*100):.1f}%)")
    else:
        print("No transactions to analyze.")
    print(f"Net Balance: ${net_balance:,.2f}")
    # Group by customer_id
    print("By customer:")
    # Initialize dictionary to hold customer totals
    customer_totals = {}
    for t in transactions:
        customer_id = t['customer_id']
        if customer_id not in customer_totals:
            customer_totals[customer_id] = {'credit': 0, 'debit': 0, 'transfer': 0}
        if t['type'] == 'credit':
            customer_totals[customer_id]['credit'] += t['amount']
        elif t['type'] == 'debit':
            customer_totals[customer_id]['debit'] += t['amount']
        elif t['type'] == 'transfer':
            customer_totals[customer_id]['transfer'] += t['amount']
    # Print customer totals
    for customer_id, totals in customer_totals.items():
        print(f"Customer {customer_id}:")
        for category in ['credit', 'debit', 'transfer']:
            try:
                amount = float(totals.get(category, 0))
            except (ValueError, TypeError):
                amount = 0
            print(f"  Total {category.title()}s: ${amount:,.2f}")
    # Print the customer with the highest debit amount
    highest_debit_customer = max(customer_totals.items(), key=lambda x: x[1]['debit'], default=(None, {'debit': 0}))
    if highest_debit_customer[0]:
        print(f"Customer with highest debit: {highest_debit_customer[0]} with ${highest_debit_customer[1]['debit']:,.2f}")
    else:
        print("No customers found.")
    return transactions

### Additional tasks to be completed
4. Save analysis to `analysis.txt`.

In [685]:
def analyze_finances(transactions, year=None):
    """
    Calculate and display financial summaries with percentages and save to file.
    
    Args:
        transactions (list): List of transaction dictionaries
        year (int, optional): Year to filter transactions by
    """
    # Create string buffer for both console and file output
    output = []
    
    # Filter transactions by year if specified
    if year:
        transactions = [t for t in transactions if t['date'].year == year]
        if not transactions:
            print(f"No transactions found for year {year}")
            return
        output.append(f"Financial Analysis for {year}")
    
    # Sum credits, debits, transfers
    total_credit = sum(t['amount'] for t in transactions if t['type'] == 'credit')
    total_debit = sum(t['amount'] for t in transactions if t['type'] == 'debit')
    total_transfer = sum(t['amount'] for t in transactions if t['type'] == 'transfer')
    net_balance = total_credit + total_debit + total_transfer
    
    # Calculate total volume (absolute sum)
    total_volume = abs(total_credit) + abs(total_debit) + abs(total_transfer)

    # Generate summary with percentages
    output.append("\nFinancial Summary:")
    if total_volume > 0:  # Avoid division by zero
        output.append(f"Total Credits: ${total_credit:,.2f} ({abs(total_credit/total_volume*100):.1f}%)")
        output.append(f"Total Debits: ${total_debit:,.2f} ({abs(total_debit/total_volume*100):.1f}%)")
        output.append(f"Total Transfers: ${total_transfer:,.2f} ({abs(total_transfer/total_volume*100):.1f}%)")
    else:
        output.append("No transactions to analyze.")
    output.append(f"Net Balance: ${net_balance:,.2f}")

    # Group by customer_id
    output.append("\nBy customer:")
    customer_totals = {}
    for t in transactions:
        customer_id = t['customer_id']
        if customer_id not in customer_totals:
            customer_totals[customer_id] = {'credit': 0, 'debit': 0, 'transfer': 0}
        if t['type'] == 'credit':
            customer_totals[customer_id]['credit'] += t['amount']
        elif t['type'] == 'debit':
            customer_totals[customer_id]['debit'] += t['amount']
        elif t['type'] == 'transfer':
            customer_totals[customer_id]['transfer'] += t['amount']

    # Add customer totals to output
    for customer_id, totals in customer_totals.items():
        customer_volume = sum(abs(amount) for amount in totals.values())
        output.append(f"\nCustomer {customer_id}:")
        for category in ['credit', 'debit', 'transfer']:
            try:
                amount = float(totals.get(category, 0))
                percentage = (abs(amount) / customer_volume * 100) if customer_volume > 0 else 0
                output.append(f"  Total {category.title()}s: ${amount:,.2f} ({percentage:.1f}%)")
            except (ValueError, TypeError):
                amount = 0
                output.append(f"  Total {category.title()}s: ${amount:,.2f} (0.0%)")

    # Add highest debit customer to output
    highest_debit_customer = max(customer_totals.items(), key=lambda x: x[1]['debit'], default=(None, {'debit': 0}))
    if highest_debit_customer[0]:
        output.append(f"\nCustomer with highest debit: {highest_debit_customer[0]} with ${highest_debit_customer[1]['debit']:,.2f}")
    else:
        output.append("\nNo customers found.")

    # Print to console and save to file
    analysis_text = '\n'.join(output)
    print(analysis_text)
    
    try:
        with open('analysis.txt', 'w') as f:
            f.write(analysis_text)
        print("\nAnalysis saved to analysis.txt")
    except IOError as e:
        print(f"\nError saving analysis to file: {e}")

    return transactions

## Task 5: Saving Transactions and Generating Reports

### For saving:

In [686]:
def save_transactions(transactions, filename='financial_transactions.csv'):
    """Save transactions to a CSV file."""
    # Open file for writing
    with open(filename, 'w') as file:
        # Use csv.DictWriter
        fieldnames = ['transaction_id', 'date', 'customer_id', 'amount', 'type', 'description']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        # Write header
        writer.writeheader()
        # Write each transaction
        for t in transactions:
            # Convert date back to string format
            t['date'] = t['date'].strftime('%Y-%m-%d')
            writer.writerow(t)
    print(f"Transactions save to {filename} successfully.")
    return transactions

### For reporting:

In [687]:
def generate_report(transactions, filename='report.txt'):
    """Generate a text report of financial summaries."""
    # Calculate metrics
    total_credit = sum(t['amount'] for t in transactions if t['type'] == 'credit')
    total_debit = sum(t['amount'] for t in transactions if t['type'] == 'debit')
    total_transfer = sum(t['amount'] for t in transactions if t['type'] == 'transfer')
    net_balance = total_credit + total_debit + total_transfer
    # Group by Type
    type_totals = {}
    for t in transactions:
        if t['type'] not in type_totals:
            type_totals[t['type']] = 0
        type_totals[t['type']] += t['amount']
    # Write to file
    with open(filename, 'w') as file:
        file.write("Financial Summary\n")
        file.write(f"Total Credits: ${total_credit:,.2f}\n")
        file.write(f"Total Debits: ${total_debit:,.2f}\n")
        file.write(f"Total Transfers: ${total_transfer:,.2f}\n")
        file.write(f"Net Balance: ${net_balance:,.2f}")
        file.write("\nBy Type:\n")
        for type, amount in type_totals.items():
            file.write(f"  {type.title()}: ${amount:,.2f}\n")
    print(f"Report generated and saved to {filename}.")
    return transactions

#### Additional tasks to be completed
1. Add a timestamp to the report filename (e.g., `report_20250508.txt`).
2. Back up the original CSV before saving.
3. Include transaction date range in the report.
4. Handle file-writing errors.

In [688]:
def generate_report(transactions):
    """Generate a text report of financial summaries with timestamped filename."""
    # Generate filename with current date
    current_date = datetime.now().strftime('%Y%m%d')
    filename = f'report_{current_date}.txt'
    
    # Get date range
    if transactions:
        start_date = min(t['date'] for t in transactions)
        end_date = max(t['date'] for t in transactions)
        date_range = f"From {start_date.strftime('%b %d, %Y')} to {end_date.strftime('%b %d, %Y')}"
    else:
        date_range = "No transactions available"
    
    # Calculate metrics
    total_credit = sum(t['amount'] for t in transactions if t['type'] == 'credit')
    total_debit = sum(t['amount'] for t in transactions if t['type'] == 'debit')
    total_transfer = sum(t['amount'] for t in transactions if t['type'] == 'transfer')
    net_balance = total_credit + total_debit + total_transfer
    
    # Group by Type
    type_totals = {}
    for t in transactions:
        if t['type'] not in type_totals:
            type_totals[t['type']] = 0
        type_totals[t['type']] += t['amount']
    
    try:
        # Write to file, handle file writing errors
        with open(filename, 'w') as file:
            file.write(f"Financial Summary (Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')})\n")
            file.write(f"Period: {date_range}\n\n")
            file.write(f"Total Credits: ${total_credit:,.2f}\n")
            file.write(f"Total Debits: ${total_debit:,.2f}\n")
            file.write(f"Total Transfers: ${total_transfer:,.2f}\n")
            file.write(f"Net Balance: ${net_balance:,.2f}")
            file.write("\nBy Type:\n")
            for type, amount in type_totals.items():
                file.write(f"  {type.title()}: ${amount:,.2f}\n")
        print(f"Report generated and saved to {filename}.")
    except IOError as e:
        print(f"Error writing report: {e}")
    
    return transactions

In [689]:
def save_transactions(transactions, filename='financial_transactions.csv'):
    """Save transactions to a CSV file with backup of original file."""
    try:
        # Create backup if original file exists
        try:
            # Generate backup filename with timestamp
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            backup_filename = f"financial_transactions_backup_{timestamp}.csv"
            
            # Read original content
            with open(filename, 'r') as source:
                content = source.read()
            
            # Write to backup file
            with open(backup_filename, 'w') as backup:
                backup.write(content)
            print(f"Backup created: {backup_filename}")
        except FileNotFoundError:
            # If original file doesn't exist, just continue with saving
            pass
        
        # Open file for writing new transactions
        with open(filename, 'w') as file:
            # Use csv.DictWriter
            fieldnames = ['transaction_id', 'date', 'customer_id', 'amount', 'type', 'description']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            # Write header
            writer.writeheader()
            # Write each transaction
            for t in transactions:
                # Convert date back to string format
                t['date'] = t['date'].strftime('%Y-%m-%d')
                writer.writerow(t)
        print(f"Transactions saved to {filename} successfully.")
    except IOError as e:
        print(f"Error saving transactions: {e}")
    
    return transactions

## Main Program: Tying it All Together

In [690]:
def main():
    transactions = []
    while True:
        print("\nSmart Personal Finance Analyzer")
        print("1. Load Transactions")
        print("2. Add Transaction")
        print("3. View Transactions")
        print("4. Update Transaction")
        print("5. Delete Transaction")
        print("6. Analyze Finances")
        print("7. Save Transactions")
        print("8. Generate Report")
        print("9. Exit")
        choice = input("Select an option: ")
        # Call functions based on choice
        try:
            if choice == '9':
                print("Thank you for using Smart Personal Finance Analyzer!")
                break
            elif choice == '1':
                transactions = load_transactions()
            elif choice == '2':
                transactions = add_transaction(transactions)
            elif choice == '3':
                view_transactions(transactions)
            elif choice == '4':
                transactions = update_transaction(transactions)
            elif choice == '5':
                transactions = delete_transaction(transactions)
            elif choice == '6':
                analyze_finances(transactions)
            elif choice == '7':
                save_transactions(transactions)
            elif choice == '8':
                generate_report(transactions)
            else:
                print("Invalid option. Please try again.")
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            continue

main()


Smart Personal Finance Analyzer
1. Load Transactions
2. Add Transaction
3. View Transactions
4. Update Transaction
5. Delete Transaction
6. Analyze Finances
7. Save Transactions
8. Generate Report
9. Exit
Number of loaded transactions: 100000

Smart Personal Finance Analyzer
1. Load Transactions
2. Add Transaction
3. View Transactions
4. Update Transaction
5. Delete Transaction
6. Analyze Finances
7. Save Transactions
8. Generate Report
9. Exit

Financial Summary:
Total Credits: $166,670,591.51 (33.3%)
Total Debits: $-166,223,463.59 (33.2%)
Total Transfers: $168,007,221.73 (33.5%)
Net Balance: $168,454,349.65

By customer:

Customer 926:
  Total Credits: $208,265.91 (38.6%)
  Total Debits: $-156,221.18 (29.0%)
  Total Transfers: $174,883.47 (32.4%)

Customer 466:
  Total Credits: $213,143.92 (36.8%)
  Total Debits: $-161,074.04 (27.8%)
  Total Transfers: $205,725.51 (35.5%)

Customer 110:
  Total Credits: $214,470.75 (39.4%)
  Total Debits: $-168,770.97 (31.0%)
  Total Transfers: $160